In [2]:
## packages
import pandas as pd
import numpy as np
import tarfile
import json

## unzip tar
# import tarfile
# tar = tarfile.open('yelp_dataset.tarf]=', 'r:gz')
# tar.extractall()
# tar.close()

In [3]:
# reviews = pd.read_json('yelp_academic_dataset_review.json', lines = True)
# users = pd.read_json('yelp_academic_dataset_user.json', lines = True)
# businesses = pd.read_json('yelp_academic_dataset_business.json', lines = True)
philly_bus = pd.read_feather('../FilteredData/business_philly.feather')
philly_reviews = pd.read_feather('../FilteredData/review_philly.feather')
philly_users = pd.read_feather('../FilteredData/user_philly.feather')

In [19]:
# business_cat = businesses.categories.unique
# df = businesses[businesses['categories'].str.contains('Restaurant')]
# philly_bus

In [5]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score,reciprocal_rank
from lightfm.data import Dataset
from lightfm import LightFM, cross_validation

c:\Users\carlee\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
df = philly_reviews.groupby('user_id')['stars'].mean()
users = pd.merge(philly_users, df, on=['user_id'], how='left')

In [7]:
bins = [0, 0.9999999, 1.9999999, 2.9999999, 3.9999999, 4.9999999, 5]
labels = ["0","1", "2", "3","4", "5"]
users["star_bin"] = pd.cut(users['stars'], bins=bins, labels=labels)
# users.loc[(users['stars'] > 4.9999999) & (users['stars'] < 5)]

In [8]:
# list(filter(lambda x: '27281' in x, uf))

In [9]:
reviews_only = philly_reviews[["user_id", "business_id", "stars"]]

In [11]:
#unique user features
user_f = []
user_col = ['star_bin']*len(users['star_bin'].unique()) 
user_unique_list = list(users['star_bin'].unique())
# col = ['review_count']*len(users['review_count'].unique()) + ['useful']*len(users['useful'].unique()) + ['funny']*len(users['funny'].unique()) + ['cool']*len(users['cool'].unique())
# unique_list = list(users['review_count'].unique()) + list(users['useful'].unique()) + list(users['funny'].unique()) + list(users['cool'].unique())


for x,y in zip(user_col, user_unique_list):
    res = str(x)+ ":" +str(y)
    user_f.append(res)
#     print(res)

In [23]:
#unique item features
item_f = []
item_col = ['stars']*len(philly_bus['stars'].unique()) + ['postal_code']*len(philly_bus['postal_code'].unique())
item_unique_list = list(philly_bus['stars'].unique()) + list(philly_bus['postal_code'].unique())

for x,y in zip(item_col, item_unique_list):
    res = str(x)+ ":" +str(y)
    item_f.append(res)
#     print(res)

In [24]:
dataset1 = Dataset()
dataset1.fit(
        philly_reviews['user_id'].unique(), # all the users
        philly_reviews['business_id'].unique(), # all the items
        user_features = user_f, # additional user features
        item_features = item_f #additional item features
)

In [25]:
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in reviews_only.values])

In [26]:
def user_feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    ll = ['star_bin:']
    # ll = ['review_count:', 'useful:', 'funny:', 'cool:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result
# Using the helper function to generate user features in proper format for ALL users
ad_subset = users[['star_bin']] 
# ad_subset = philly_users[["review_count",'useful', 'funny', 'cool']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(user_feature_colon_value(item))
# print(f'Final output: {feature_list}')
user_tuple = list(zip(users.user_id, feature_list))
user_features = dataset1.build_user_features(user_tuple, normalize= False)


In [14]:
def item_feature_colon_value(my_list):
    result = []
    ll = ['review_count:', 'useful:', 'funny:', 'cool:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result
# Using the helper function to generate user features in proper format for ALL users
ad_subset = users[['star_bin']] 
# ad_subset = philly_users[["review_count",'useful', 'funny', 'cool']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(item_feature_colon_value(item))
# print(f'Final output: {feature_list}')
item_tuple = list(zip(philly_bus.business_id, feature_list))
item_features = dataset1.build_user_features(item_tuple, normalize= False)


NameError: name 'feature_list' is not defined

In [ ]:
train, test = cross_validation.random_train_test_split(interactions, test_percentage=0.25)

In [ ]:
model = LightFM()
model.fit(train,
      epochs=30,
      num_threads=4)

In [ ]:
from lightfm.evaluation import auc_score
test_auc = auc_score(model,
                      test,
                     ).mean()
print('Hybrid training set AUC: %s' % test_auc)

Hybrid training set AUC: 0.8455258


In [ ]:
model2 = LightFM() #loss='warp'
model2.fit(train,
      user_features= user_features,
      # item_features = item_features
      epochs=30,
      num_threads=4)

In [ ]:
test_auc2 = auc_score(model2,
                      test,
                      user_features=user_features
                     ).mean()
print('Hybrid training set AUC: %s' % test_auc2)

Hybrid training set AUC: 0.81536126
